#### QGIS -> demand data

In [16]:
import pandas as pd, networkx as nx, math, os

node_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\1.NODE_export.xlsx"
edge_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\1.EDGE_export.xlsx"
out_path  = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_data_final.xlsx"

edge_df = pd.read_excel(edge_path)
node_df = pd.read_excel(node_path)

def pt(s):
    a=s.replace('Point (','').replace(')','').split(); return float(a[0]),float(a[1])

coord = {str(r['node_id']): pt(r['geometry']) for _,r in node_df.iterrows()}

kor_name = {}
for _,r in edge_df.iterrows():
    kor_name[str(r['from_node_'])]=r['from_nod_1']
    kor_name[str(r['to_node_id'])]=r['to_node_nm']

def segdist(a,b):
    if a not in coord or b not in coord: return None
    (x1,y1),(x2,y2)=coord[a],coord[b]; return math.hypot(x1-x2,y1-y2)

rows=[]
for line,g in edge_df.groupby('RLWAY_NM'):
    G=nx.DiGraph()
    for _,r in g.iterrows():
        G.add_edge(str(r['from_node_']),str(r['to_node_id']))
    for comp in nx.weakly_connected_components(G):
        sg=G.subgraph(comp)
        try: path=list(nx.topological_sort(sg))
        except nx.NetworkXUnfeasible: continue
        for seq in (path, path[::-1]):
            n=len(seq)
            for i in range(n):
                for j in range(i+1,n):
                    sub=seq[i:j+1]
                    d=sum(segdist(sub[k],sub[k+1]) for k in range(len(sub)-1))
                    rows.append([
                        line,
                        sub[0],sub[-1],
                        '-'.join(sub),
                        f"n{sub[0]}",f"n{sub[-1]}",
                        kor_name.get(sub[0]),kor_name.get(sub[-1]),
                        d
                    ])

cols=['line','origin','destination','node_path','origin_name',
      'destination_name','origin_kor_name','destination_kor_name','distance']
pd.DataFrame(rows,columns=cols).drop_duplicates().to_excel(out_path,index=False)
print('saved:',os.path.basename(out_path))


saved: demand_data_final.xlsx


#### 역사별 승하차데이터로 임시 journeys demand data 만들기

In [24]:
import pandas as pd
from pathlib import Path

# 디렉터리 경로
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")

# 1) 역명 → RLNODE_NM 생성
ridership_path = base_dir / "3.station_ridership.xlsx"
ridership_df = pd.read_excel(ridership_path)

ridership_df["RLNODE_NM"] = ridership_df["역명"].str.strip() + "역"   # 공백 제거 후 '역' 붙이기

# 2) 수요 데이터에서 고유 역 이름 추출
#   → 2.demand_data_final 의 실제 확장자를 지정하세요 (.xlsx, .csv 등)
demand_path = base_dir / "2.demand_data_final.xlsx"   # 예: .xlsx인 경우
demand_df = pd.read_excel(demand_path)

unique_names = pd.unique(                  # 중복 제거
    pd.concat([
        demand_df["origin_kor_name"].astype(str).str.strip(),
        demand_df["destination_kor_name"].astype(str).str.strip()
    ])
)

# 3) 수요에 등장하는 역만 필터링
filtered_df = ridership_df[ridership_df["RLNODE_NM"].isin(unique_names)]

# 4) 결과 저장
output_path = base_dir / "4.station_ridership_90node.xlsx"
filtered_df.to_excel(output_path, index=False)

print(f"✔️ 완료: {len(filtered_df)}개 역을 '{output_path.name}'에 저장했습니다.")


✔️ 완료: 83개 역을 '4.station_ridership_90node.xlsx'에 저장했습니다.


In [25]:
import pandas as pd
from pathlib import Path

# 디렉토리 설정
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")

# 파일 경로
node_path = base_dir / "1.NODE_export.xlsx"
ridership_90_path = base_dir / "4.station_ridership_90node.xlsx"

# 파일 읽기
node_df = pd.read_excel(node_path)
ridership_90_df = pd.read_excel(ridership_90_path)

# RLNODE_NM 열 기준으로 차집합 계산
all_nodes = set(node_df["RLNODE_NM"].dropna().astype(str).str.strip())
included_nodes = set(ridership_90_df["RLNODE_NM"].dropna().astype(str).str.strip())

# NODE_export에만 있고, ridership_90에는 없는 값
missing_nodes = sorted(all_nodes - included_nodes)

# 결과 출력
print(f"✅ NODE_export에는 있지만 ridership_90에는 없는 역 개수: {len(missing_nodes)}")
print("누락된 역 목록:", ', '.join(missing_nodes))


✅ NODE_export에는 있지만 ridership_90에는 없는 역 개수: 7
누락된 역 목록: 경기광주역, 삼척역, 신경주역, 여주역, 인천역, 천안아산역(온양온천), 판교역


#### Journeys data 추가

In [ ]:
import pandas as pd
import numpy as np

base_dir = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"

demand = pd.read_excel(f"{base_dir}\\2.demand_data_final.xlsx")
stations = pd.read_excel(f"{base_dir}\\4.station_ridership_90node.xlsx",
                         usecols=['RLNODE_NM', '승차인원', '하차인원'])

demand = demand.merge(
    stations.rename(columns={'RLNODE_NM': 'origin_kor_name', '승차인원': 'O_i'}),
    on='origin_kor_name', how='left')

demand = demand.merge(
    stations.rename(columns={'RLNODE_NM': 'destination_kor_name', '하차인원': 'D_j'}),
    on='destination_kor_name', how='left')

demand['dist_km'] = demand['distance'] / 1000

beta = 1.0
demand['w_ij'] = demand['D_j'] / (demand['dist_km'] ** beta)
demand['sum_w_i'] = demand.groupby('origin_kor_name')['w_ij'].transform('sum')
demand['journeys'] = demand['O_i'] * (demand['w_ij'] / demand['sum_w_i'])

demand = demand.drop(columns=['w_ij', 'sum_w_i'])
demand.to_excel(f"{base_dir}\\2.demand_data_with_journeys.xlsx", index=False)
print("✅ Saved successfully → 2.demand_data_with_journeys.xlsx")

저장 완료 → 2.demand_data_with_journeys.xlsx


#### Convert to demand_data.json

In [ ]:
import pandas as pd
import json
from pathlib import Path

base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
excel_path = base_dir / "2.demand_data_with_journeys.xlsx"
json_path  = base_dir / "5.demand_data_kofull.json"

cols = [
    "origin", "destination",
    "distance", "journeys",
    "origin_name", "destination_name"
]

df = pd.read_excel(excel_path, usecols=cols)
df["distance"] = df["distance"].astype(float)
df["journeys"] = df["journeys"].astype(float)

records = df.to_dict(orient="records")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Saved successfully → {json_path}")

✅ Saved successfully → D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_data_kofull.json
